
# BE Meteo 
## HERVIOUX, SOURDEVAL, MANCIET, DUC-MARTIN, LEFAUCONNIER

In [ ]:
# On load les données grâce à un fichier txt qui contient le chemin vers le fichier csv contenant les données
path_data <- paste(readLines("path.txt"), collapse = "\n")

In [ ]:
#install.packages("data.table")
library(data.table)
dataset_meteo <- fread(path_data)
head(dataset_meteo)

# 1 - Description des données

### 1.1 - Organisation des données par ordre chronologique

In [ ]:
df_meteo <- data.frame(dataset_meteo)
df_meteo <- df_meteo[order(df_meteo$Date), ] # Trier les valeurs par ordre chronologique
head(df_meteo)

### 1.2 - Description des paramètres

In [ ]:
cat("Nombre de lignes de données :", nrow(df_meteo), "\n")
cat("Nombre de paramètres :", ncol(df_meteo), "\n")
cat("\tdont 14 paramètres décrivant l'emplacement de la station météo\n")
cat("\tdont 19 paramètres relatifs aux nuages/à la nébulosité (+ 4 hauteurs de bases)\n")
cat("\tdont 14 paramètres relatifs à la température\n")
cat("\tdont 7 paramètres relatifs à la pression/barométrie\n")
cat("\tdont 5 paramètres relatifs au vent/rafales\n")
cat("\tdont 5 paramètres relatifs à la pluie (précipitations)\n")
cat("\tdont 3 paramètres relatifs à la neige\n")
cat("\tdont 2 paramètres relatifs à l'humidité/rosée\n")
cat("\t + Date, mois_de_l_annee, Visibilité.horizontale, Temps.présent, Temps.passé.1, Temps.passé.2, Géopotentiel, Etat.du.sol, Phénomène.spécial.1, Phénomène.spécial.2, Phénomène.spécial.3, Phénomène.spécial.4, Temps.passé.1.1, Temps.présent.1")

### 1.3 - Description des stations météo

In [ ]:
# Caractéristiques des stations météo
df_stations <- df_meteo[c('ID.OMM.station', 'Coordonnees', 'Nom', 'Latitude', 'Longitude', 'Altitude', 'communes..name.', 'communes..code.', 'EPCI..name.', 'EPCI..code.', 'department..name.', 'department..code.', 'region..name.', 'region..code.')]
df_unique_stations <- df_stations[!duplicated(df_stations[c('ID.OMM.station')]), ]
cat("Nombre de stations météo :", nrow(df_unique_stations))
head(df_unique_stations[order(df_unique_stations$Nom), ])

### 1.4 - Carte des stations météo

In [ ]:
#install.packages(c("ggplot2", "maps", "ggmap"))
library(ggplot2)
library(maps)
library(ggmap)

options(repr.plot.width = 20, repr.plot.height = 10)

data_points <- data.frame(
  lon = df_unique_stations$Longitude,
  lat = df_unique_stations$Latitude,
  Ville = df_unique_stations$Nom
)

world <- map_data("world")

ggplot() +
  geom_polygon(data = world, aes(x = long, y = lat, group = group),
               fill = "lightgray", color = "white") +
  geom_point(data = data_points, aes(x = lon, y = lat, color = Ville), size = 1) +
  theme_minimal() +
  theme(legend.position = "none") +
  ggtitle("Carte des stations météo")


# Zoom sur l'hexagone  
data_points_metropole <- data_points[(data_points$lat<=52)&(data_points$lat>=42)&(data_points$lon<=10)&(data_points$lon>=-10),]
france <- map_data("france")

ggplot() +
  geom_polygon(data = france, aes(x = long, y = lat, group = group),
               fill = "lightgray", color = "white") +
  geom_point(data = data_points_metropole, aes(x = lon, y = lat, color = Ville), size = 3) +
  theme_minimal() +
  # theme(legend.position = "none") +
  coord_fixed(ratio = 1.2)+
  ggtitle("Carte des stations météo de France métropolitaine")

### 1.5 - Relevé de températures par station

In [ ]:
par(bg='white')
for (ville in unique(df_meteo$Nom)[1:3]){
  plot(
    df_meteo$Date[df_meteo$Nom==ville],
    df_meteo$`Température`[df_meteo$Nom==ville],
    xlab = "Date",
    ylab = "Température",
    main = ville,
    type = "l",
)}

### 1.6 - Valeurs moyennes des grandeurs mesurées par station

In [ ]:
# install.packages("dplyr")
library(dplyr)

df_grouped <- df_meteo %>%
  group_by(ID.OMM.station) %>%
  summarise(n_mesures = n(),
            T_mean = mean(`Température`,na.rm = TRUE),
            T_min = min(`Température`,na.rm = TRUE),
            T_max = max(`Température`,na.rm = TRUE),
            P_mean = mean(Pression.au.niveau.mer,na.rm = TRUE),
            nebulosite_mean = mean(Nebulosité.totale, na.rm = TRUE)
            )

head(df_grouped)

### 1.7 - Carte des températures extrêmes mesurées par station

In [ ]:
df_station_stats <- merge(x = df_unique_stations, y = df_grouped, by = "ID.OMM.station", all = TRUE)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)

world <- map_data("world")

ggplot() +
  geom_polygon(data = world, aes(x = long, y = lat, group = group),
               fill = "lightgray", color = "white") +
  geom_point(data = df_station_stats, aes(x = Longitude, y = Latitude, color = T_min), size = 1) +
  theme_minimal() +
  # theme(legend.position = "none")+
  ggtitle("Température minimale par station (K)")

ggplot() +
  geom_polygon(data = world, aes(x = long, y = lat, group = group),
               fill = "lightgray", color = "white") +
  geom_point(data = df_station_stats, aes(x = Longitude, y = Latitude, color = T_max), size = 1) +
  theme_minimal() +
  # theme(legend.position = "none")+
  ggtitle("Température maximale par station (K)")

In [ ]:
summary(dataset_meteo)

# 2. Un calcul et étude d'estimateur

Nous allons étudier 3 estimateurs sur le jeu de donnée météo :

### 2.1 - $\small\textbf{Estimateur 1 : }$

Etude d'un estimateur de la température moyenne sur l'hiver 2019 en France métropolitaine.

In [ ]:
#On extrait les données utiles du dataset
data_points <- data.frame(
  date = df_meteo$Date,
  temp = df_meteo$`Température...C.`,
  station = df_meteo$Nom,
  mois = df_meteo$mois_de_l_annee,
  lat = df_meteo$Latitude,
  long = df_meteo$Longitude
)

france <- map_data("france")

#on filtre le dataset pour avoir les données voulues et calculer la moyenne de température
data_points_metropole <- data_points %>%
  filter(lat <= 52, lat >= 42,
         long <= 10, long >= -10,
         date < "2020-01-01", date > "2019-01-01",
         mois > 1, mois < 4) %>%
  group_by(station, lat, long) %>%
  summarise(temp_moy = mean(temp, na.rm = TRUE), .groups = "drop")

#plot de la carte obtenue
ggplot() +
  geom_polygon(data = france, aes(x = long, y = lat, group = group),
               fill = "lightgray", color = "white") +
  geom_point(data = data_points_metropole, aes(x = long, y = lat, color = temp_moy), size = 3) +
  theme_minimal() +
  scale_color_gradient(low = "blue", high = "red",
                     name = "Température moyenne (°C)") +
  coord_fixed(ratio = 1.2)+
  ggtitle("Moyenne des température sur l'hiver 2019")


Grâce à cette carte, nous pouvons voir la répartition spatiale de la température moyenne de l'hiver 2019. Nous observons que les zones géographiques au Sud sont plus chaudes que celles au Nord Est. Nous pouvons donc observer un gradient de température du Sud vers le Nord ce qui correspond empiriquement à l'idée que nous avons de la météo.

In [ ]:
library(dplyr)
library(lubridate)
library(ggplot2)

#On extrait les données utiles du dataset
data_points <- data.frame(
  date = df_meteo$Date,
  temp = df_meteo$`Température...C.`,
  station = df_meteo$Nom,
  mois = df_meteo$mois_de_l_annee,
  lat = df_meteo$Latitude,
  long = df_meteo$Longitude
)

# Paramètre : niveau de confiance
alpha <- 0.05  # pour IC à 95%

# Calculer par année : moyenne, sd, n et IC
ic_par_annee <- data_points %>%
  filter(lat <= 52, lat >= 42,
         long <= 10, long >= -10,
         date < "2020-01-01", date > "2019-01-01",
         mois > 1, mois < 4) %>%
  group_by(station, lat, long) %>%
  summarise(
    n = sum(!is.na(temp)),
    temp_moy = mean(temp, na.rm = TRUE),
    sd_temp = sd(temp, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  filter(n > 1) %>%
  mutate(
    se = sd_temp / sqrt(n),    # erreur standard
    ic_bas = temp_moy - sqrt(se / (alpha * n)),
    ic_haut = temp_moy + sqrt(se / (alpha * n))
  )


# Graphe avec barres d'erreur
ggplot(ic_par_annee, aes(x = station, y = temp_moy)) +
  geom_line() +
  geom_point(size = 2) +
  geom_errorbar(aes(ymin = ic_bas, ymax = ic_haut), width = 0.2) +
  labs(
    title = paste0("Température moyenne annuelle et Intervalle de Confiance à ", 100*(1-alpha), "%"),
    x = "Station",
    y = "Température moyenne (°C)"
  ) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1)) 


L'estimateur de la moyenne est un estimateur sans biais par construction. Nous avons donc calculé l'intervalle de confiance à 95% de cet estimateur pour chacune des stations qui dépend de l'estimateur de la variance. Nous observons un intervalle de confiance qui est plutôt précis par rapport à la valeur absolue de la moyenne. Ceci est dû au fait que nous avons beaucoup de données et donc l'intervalle de confiance est réduit.

D'après la formule : 
$
IC = \left[ \; \bar{X}_n - \sqrt{\frac{\hat{\sigma}_n^2}{\alpha n}} \; , \; 
              \bar{X}_n + \sqrt{\frac{\hat{\sigma}_n^2}{\alpha n}} \; \right]
$

### 2.2 - $\small\textbf{Estimateur 2 : }$

Nous allons faire de même avec une estimation des précipitations moyennes en France métropolitaine sur l'hiver 2019.

In [ ]:
#On extrait les données utiles du dataset
data_points <- data.frame(
  date = df_meteo$Date,
  pluie = df_meteo$`Précipitations.dans.la.dernière.heure`,
  station = df_meteo$Nom,
  mois = df_meteo$mois_de_l_annee,
  lat = df_meteo$Latitude,
  long = df_meteo$Longitude
)

france <- map_data("france")

#on filtre le dataset pour avoir les données voulues et calculer la moyenne de température
data_points_metropole <- data_points %>%
  filter(lat <= 52, lat >= 42,
         long <= 10, long >= -10,
         date < "2020-01-01", date > "2019-01-01",
         mois > 1, mois < 4) %>%
  group_by(station, lat, long) %>%
  summarise(pluie_moy = mean(pluie, na.rm = TRUE), .groups = "drop")

#plot de la carte obtenue
ggplot() +
  geom_polygon(data = france, aes(x = long, y = lat, group = group),
               fill = "lightgray", color = "white") +
  geom_point(data = data_points_metropole, aes(x = long, y = lat, color = pluie_moy), size = 3) +
  theme_minimal() +
  scale_color_gradient(low = "blue", high = "red",
                     name = "Précipitation (mm)") +
  coord_fixed(ratio = 1.2)+
  ggtitle("Moyenne des précipitations sur l'hiver 2019")


Grâce à cette carte et à l'estimation des précipitations, nous pouvons observer les régions où il a le plus plu durant l'hiver 2019. Nous observons que la Bretagne est une région où il a particulièrement plu durant cet hiver ce qui confirme aussi la croyance populaire...

In [ ]:
library(dplyr)
library(lubridate)
library(ggplot2)

#On extrait les données utiles du dataset
data_points <- data.frame(
  date = df_meteo$Date,
  pluie = df_meteo$`Précipitations.dans.la.dernière.heure`,
  station = df_meteo$Nom,
  mois = df_meteo$mois_de_l_annee,
  lat = df_meteo$Latitude,
  long = df_meteo$Longitude
)

# Paramètre : niveau de confiance
alpha <- 0.05  # pour IC à 95%

# Calculer par année : moyenne, sd, n et IC
ic_par_annee <- data_points %>%
  filter(lat <= 52, lat >= 42,
         long <= 10, long >= -10,
         date < "2020-01-01", date > "2019-01-01",
         mois > 1, mois < 4) %>%
  group_by(station, lat, long) %>%
  summarise(
    n = sum(!is.na(pluie)),
    pluie_moy = mean(pluie, na.rm = TRUE),
    sd_pluie = sd(pluie, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  filter(n > 1) %>%
  mutate(
    se = sd_pluie / sqrt(n),    # erreur standard
    ic_bas = pluie_moy - sqrt(se / (alpha * n)),
    ic_haut = pluie_moy + sqrt(se / (alpha * n))
  )


# Graphe avec barres d'erreur
ggplot(ic_par_annee, aes(x = station, y = pluie_moy)) +
  geom_line() +
  geom_point(size = 2) +
  geom_errorbar(aes(ymin = ic_bas, ymax = ic_haut), width = 0.2) +
  labs(
    title = paste0("Précipations moyennes annuelles et Intervalle de Confiance à ", 100*(1-alpha), "%"),
    x = "Station",
    y = "Précipitation moyenne (°C)"
  ) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1)) 


Nous avons aussi tracé l'intervalle de confiance pour l'estimateur de la précipitation moyenne. Nous observons une plus grande disparité dans les données de par leur variance plus grande et leur plus faible nombre par rapport à la température vu plus tôt. Nous pouvons donc en déduire que l'estimateur de la précipitation moyenne est plus suceptible d'être moins précis que celui de la température.

### 2.3 - $\small\textbf{Estimateur 3 : }$

Nous allons finir par estimer la température moyenne en France métropolitaine sur une année.

In [ ]:
#On extrait les données utiles du dataset
data_points <- data.frame(
  date = df_meteo$Date,
  temp = df_meteo$`Température...C.`,
  lat = df_meteo$Latitude,
  long = df_meteo$Longitude
)

#on filtre le dataset pour avoir les données voulues et calculer la moyenne de température
data_points <- data_points%>%
  mutate(annee = year(date))
data_points <- data_points %>%
  group_by(annee) %>%
  summarise(temp_moy = mean(temp, na.rm = TRUE), .groups = "drop")

#plot du graphe obtenu
ggplot(data_points, aes(x = annee, y = temp_moy)) +
  geom_line(color = "blue", size = 1) +
  geom_point(color = "red", size = 2) +
  labs(
    title = "Évolution de la température moyenne annuelle",
    x = "Année",
    y = "Température moyenne (°C)"
  ) +
  theme_minimal()

Grâce à ce graphe nous étudions l'évolution de la température moyenne annuelle de 2015 à 2025. Nous observons une tendance croissante de cette température moyenne annuelle. Nous confirmons donc un des effets du réchauffement climatique :  l'augmentation de la température en France métropolitaine depuis 10 ans. 

In [ ]:
library(dplyr)
library(lubridate)
library(ggplot2)

# Paramètre : niveau de confiance
alpha <- 0.05  # pour IC à 95%

#On extrait les données utiles du dataset
data_points <- data.frame(
  date = df_meteo$Date,
  temp = df_meteo$`Température...C.`,
  lat = df_meteo$Latitude,
  long = df_meteo$Longitude
)

# Calculer par année : moyenne, sd, n et IC t
ic_par_annee <- data_points %>%
  mutate(annee = year(date)) %>%
  group_by(annee) %>%
  summarise(
    n = sum(!is.na(temp)),
    temp_moy = mean(temp, na.rm = TRUE),
    sd_temp = sd(temp, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  mutate(
    se = sd_temp / sqrt(n),
    ic_bas = temp_moy - sqrt(se / (alpha * n)),
    ic_haut = temp_moy + sqrt(se / (alpha * n))
  )

# Graphe avec barres d'erreur
ggplot(ic_par_annee, aes(x = annee, y = temp_moy)) +
  geom_line() +
  geom_point(size = 2) +
  geom_errorbar(aes(ymin = ic_bas, ymax = ic_haut), width = 0.2) +
  labs(
    title = paste0("Température moyenne annuelle et IC (", 100*(1-alpha), "%)"),
    x = "Année",
    y = "Température moyenne (°C)"
  ) +
  theme_minimal()

Nous avons retracé le même graphe qu'au dessus en ajoutant les intervalles de confiance. Dû au grand nombre de données sur l'année, nous avons un intervalle de confiance très petit. Nous sommes donc ici sur un estimateur très précis.

# 3. Des tests statistiques (au moins un sur un paramètre et une ANOVA)

Colin

In [ ]:
# on reprend : dataset_meteo
colnames(dataset_meteo)

In [ ]:

#On extrait les donnée (colonnes) utiles du dataset
data_points <- data.frame(
  date = dataset_meteo$Date,
  station = dataset_meteo$Nom,
  lat = dataset_meteo$Latitude,
  long = dataset_meteo$Longitude,
  temp = dataset_meteo$'Température (°C)',
  humidity = dataset_meteo$Humidité,
  precipitations24 = dataset_meteo$'Précipitations dans les 24 dernières heures'
)

class(data_points)
head(data_points)
nrow(data_points)


In [ ]:
df_2020 <- data_points[format(data_points$date, "%Y") == "2020", ]
nrow(df_2020)

In [ ]:
par(bg = "white")
hist(df_2020$temp,
     main = "Histogramme de la température",
     xlab = "temp",
     col = "lightblue",
     border = "black")
abline(v = mean(df_2020$temp, na.rm = TRUE), col = "red", lwd = 2, lty = 2)
print("moyenne = ")
print(mean(df_2020$temp, na.rm = TRUE))

### ANOVA
On s'intéresse aux valeurs de températures, de précipitations et humidité pour différentes régions.

# 4. Une régression et étude de corrélation

### Régression
On s'intéresse aux valeurs de pression mesurées au niveau de la mer et au niveau de la station.

On s'attend en effectuant des régressions linéaires à retrouver un modèle similaire au modèle physique de pression (statique des fluides).

$P(h) = P_{h=0} + \rho * g * h$

- $\rho$ masse volumique air en $kg/m^3$
- $g$ accélération de la pesanteur $9.81m/s²$
- $h$ hauteur en $m$

In [ ]:
n_graph_max <- 5

df_regression_results = data.frame(matrix(ncol = 4, nrow = 0))
col_names <- c("Nom","(Intercept)","Pression.au.niveau.mer", "R2")
names(df_regression_results) <- col_names

i <- 0
for (station in sort(df_station_stats$Nom)){
    if (station!='EMBRUN' & station!="LE PUY-LOUDES"){
        df_meteo_station <- df_meteo[df_meteo$Nom==station, c('ID.OMM.station', 'Date', 'Pression.au.niveau.mer', 'Variation.de.pression.en.24.heures', 'Variation.de.pression.en.3.heures', 'Précipitations.dans.la.dernière.heure', 'Précipitations.dans.les.3.dernières.heures', 'Précipitations.dans.les.24.dernières.heures', 'Type.de.tendance.barométrique', 'Point.de.rosée', 'Humidité', 'Pression.station', 'Niveau.barométrique','Altitude')]
        model_pression <- lm(Pression.station ~ Pression.au.niveau.mer, data=df_meteo_station)
        # cat("R² : ",summary(model_pression)$r.squared)
        if (i<n_graph_max){
            par(bg='white')
            plot(x=df_meteo_station$Pression.au.niveau.mer, y=df_meteo_station$Pression.station, main=station)
            abline(model_pression)
            
        }
        i <- i+1

        df_station_results<-data.frame(station,model_pression$coefficients['(Intercept)'], model_pression$coefficients['Pression.au.niveau.mer'], summary(model_pression)$r.squared,row.names = station)
        names(df_station_results)<- col_names

        df_regression_results <- rbind(df_regression_results, df_station_results)}
    }

In [ ]:
df_regression_results

In [ ]:
cat("Coefficient directeur moyen des régressions linéaires :",mean(df_regression_results$Pression.au.niveau.mer))

Les régressions linéaires effectuées semblent confirmer la cohérence d'un modèle $P(h) = P_{h=0} + cste$.

Il est difficile de conclure et confirmer la modélisation en $\rho gh$ de l'intercept, la valeur de $\rho$ dépendant de la température de manière non linéaire (on a un terme qui fait intervenir à la fois l'altitude $h$ et la température $T$).

### ANOVA

On veut tester si les stations ont les mêmes moyennes pour les variables quantitatives telles que la vitesse du vent moyen 10min.

Une ANOVA requiert l'indépendance entre les variables aléatoires et que ces variables aléatoires suivent des lois normales de même variance. On va donc considérer la vitesse du vent moyen au pas 10min car ces moyennes font intervenir des mesures d'un unique pas de temps dans l'échantillonage des données météo (3 heures).
On fait l'hypothèse que chaque mesure sera indépendante des autres (pas de recouvrement des mesures (10min<3h), mais le fait qu'on ait une série temporelle peut remettre en cause cette supposée indépendance).

In [ ]:
par(bg='white')
boxplot(Vitesse.du.vent.moyen.10.mn ~ Nom, data = df_meteo)

In [ ]:
cat("ANOVA Vitesse du vent moyen 10min\n")
aov_vitesse_vent_10mn = aov(Vitesse.du.vent.moyen.10.mn ~ Nom, df_meteo)
print(aov_vitesse_vent_10mn)
summary(aov_vitesse_vent_10mn)

Le test statistique de l'ANOVA (test de Fisher) donne une valeur de 14044 ce qui correspond à une p-value inférieure à $2e-16$. Pour un $\alpha=0.001$, on rejette l'hypothèse $H_0$ qui dit que toutes les moyennes des mesures de vitesse de vent par groupe de station sont égales (intervalle de confiance à 99,9%).

Cela signifie que certaines stations météo ont sont exposées en moyenne (10min) un vent plus rapide que d'autres stations. En effet, on remarque déjà dans le boxplot ci-dessus que certains territoires ont des distributions de vitesse de vent assez différentes des autres (notamment pour les territoires insulaires ou du littoral tels que DUMONT D'URVILLE, CROZET, JUAN DE NOVA, KERGUELEN ou encore NOUVELLE AMSTERDAM).

# 5. Une ACP et une PLS

Nuages/nébulosité

Adrien

In [ ]:
sub <- df_meteo[df_meteo$Date > "2021-01-01 00:00:00 UTC" & df_meteo$Date <= "2021-12-31 00:00:00 UTC" & df_meteo$Nom == "TOULOUSE-BLAGNAC", ]

In [ ]:
var <- c(
  "Température...C.", "Humidité", "Pression.station",
  "Visibilité.horizontale", "Précipitations.dans.les.24.dernières.heures",
  "Nebulosité.totale", "Hauteur.de.la.base.des.nuages.de.l.étage.inférieur"
)

X_focus <- sub[, var]
X_focus <- na.omit(X_focus)

library(FactoMineR)

res_pca_focus <- PCA(X_focus, scale.unit = TRUE, graph = TRUE)
# res_pca_focus$eig
# res_pca_focus$var

$\small\textbf{Lancement de l'ACP}$

Nous allons utiliser la librairie factominer. 

In [ ]:
par(bg='white')
var.coord <- res_pca_focus$var$coord[,1:2]
plot(var.coord, type='n',xlim=c(-2,2), ylim=c(-2,2), xlab="PC1", ylab = "PC2")
symbols(0,0, circles=2, inches=FALSE, add=TRUE, lwd=2)
arrows(0,0, var.coord[,1]*2, var.coord[,2]*2, length=0.1, col='red')
text(var.coord[,1]*1.1, var.coord[,2]*1.1, labels = rownames(var.coord), col ="red", cex = 0.6)
abline(h=0, v=0, col='grey70')

In [ ]:
par(bg='white')
ind.coord <- res_pca_focus$ind$coord[,1:2]
plot(ind.coord, xlab="PC1", ylab="PC2", main="Individus - focus nuages", pch=19, col="blue")

In [ ]:
# Coordonnées
ind.coord <- res_pca_focus$ind$coord[,1:2]  # individus
var.coord <- res_pca_focus$var$coord[,1:2]  # variables

par(bg='white')
# 1. Base plot
plot(ind.coord[,1], ind.coord[,2],
     xlab="PC1", ylab="PC2", main="Biplot ACP",
     pch=19, col=rgb(0,0,1,0.5))  # individus en bleu transparent
abline(h=0,v=0,col="grey80")

# 2. Cercle de corrélation
symbols(0,0, circles=2, inches=FALSE, add=TRUE, lwd=2, fg="grey50")

# 3. Vecteurs des variables
arrows(0,0,var.coord[,1]*2,var.coord[,2]*2,length=0.1,col="black", lwd=1.5)
text(var.coord[,1]*1.1, var.coord[,2]*1.1,
     labels=rownames(var.coord), col="black", cex=0.4)


$\small\textbf{Conclusion sur l'ACP}$

Nous remarquons que sur notre jeu de données, il y a une forte corrélation entre l'humidité, la hauteur des nuages et la visibilité. Cela fait sens : plus les nuages sont haut, plus la visibilité est élevée (corrélation positive).
La corrélation négative entre l'humidité et la hauteur des nuages et la visibilité est moins évidente. 
Enfin, la pression semble être corrélée négativement avec les précipitations, et ces deux variables sont orthogonales aux variables sur la nébulosité et l'humidité. 

$\small\textbf{Mise en PLS}$

La PLS permet de sélectionner les variables qui expliquent le mieux la variance d'une autre variable. Ici, nous avons voulu trouver les variables qui expliquent le mieux la nébulosité totale sur un station météo. Pour ce faire, nous avons effectué la PLS sur la nébulosité totale avec la température, l'humidité, la hauteur des nuages et la pression à la station en variables explicatives. 

In [ ]:
install.packages("pls")
library(pls)


In [ ]:
# "Température...C.", "Humidité", "Pression.station",
#   "Visibilité.horizontale", "Précipitations.dans.les.24.dernières.heures",
#   "Nebulosité.totale", "Hauteur.de.la.base.des.nuages.de.l.étage.inférieur"
# Exemple : prédire précipitations
X <- sub[, c("Température...C.", "Pression.station", "Humidité","Hauteur.de.la.base.des.nuages.de.l.étage.inférieur" )]
Y <- sub$`Nebulosité.totale`

# Modèle PLS
mod_pls <- plsr(Y ~ ., data=data.frame(X, Y), scale=TRUE, validation="CV")

# Résumé
summary(mod_pls)
loadings(mod_pls)
# Graphiques utiles
plot(RMSEP(mod_pls))   # erreur de prédiction en fonction du nb de composantes
plot(mod_pls, plottype="scores", comps=1:2)  # individus dans l'espace PLS


# 6. Une méthode de clustering

Nous souhaitons effectuer un clustering sur les stations météos. Pour cela, nous créons un data frame qui contient une ligne par station météo de France métropolitaine. Dans cette data frame, nous allons mettre les valeurs moyennes de température, humidité, pression, direction et vitesse moyenne du vent par mois entre janvier 2016 et décembre 2024, pour chaque station.

In [ ]:
data_stations <- data.frame(
  id = df_meteo$ID.OMM.station,
  lon = df_meteo$Longitude,
  lat = df_meteo$Latitude,
  Ville = df_meteo$Nom,
  date = df_meteo$Date,
  temp = df_meteo$Température,
  hum = df_meteo$`Humidité`,
  pression = df_meteo$Pression.station,
  dir_vent = df_meteo$Direction.du.vent.moyen.10.mn,
  vit_vent = df_meteo$Vitesse.du.vent.moyen.10.mn,
  mois = df_meteo$mois_de_l_annee
)

data_stations_metropole <- data_stations[(data_stations$lat<=52)&(data_stations$lat>=42)&(data_stations$lon<=10)&(data_stations$lon>=-10),]

coord_stations <- data_stations_metropole %>%
  select(id, lon, lat, Ville) %>%
  distinct(id, .keep_all = TRUE)

#head(data_stations_metropole)
mois_par_trimestre <- matrix(1:12, ncol=4, byrow = TRUE)
df_bystations <- NULL

#for (annee_encours in 2016:2024){
annee_encours <- 2017

  for(trimestre_encours in 1:3){
# Filtrer et calculer la moyenne de température par station
    moyennes_par_station <- data_stations_metropole %>%
      filter(format(date,"%Y")==annee_encours, mois %in% as.vector(mois_par_trimestre[trimestre_encours, ])) %>%
      group_by(id) %>%
      summarise(temp_moyenne = mean(temp, na.rm = TRUE),
                hum_moyenne = mean(hum, na.rm = TRUE),
                pression_moyenne = mean(pression, na.rm = TRUE)) %>%
      ungroup()
      # Création du nom de la colonne
    nom_colonne_temp <- paste0("temp_T", trimestre_encours, "_", annee_encours)
    nom_colonne_hum <- paste0("hum_T", trimestre_encours, "_", annee_encours)
    nom_colonne_pression <- paste0("pression_T", trimestre_encours, "_", annee_encours)
    # Renommer la colonne de température moyenne
    colnames(moyennes_par_station)[colnames(moyennes_par_station) == "temp_moyenne"] <- nom_colonne_temp
    colnames(moyennes_par_station)[colnames(moyennes_par_station) == "hum_moyenne"] <- nom_colonne_hum
    colnames(moyennes_par_station)[colnames(moyennes_par_station) == "pression_moyenne"] <- nom_colonne_pression
    # Fusion avec le dataframe final
    if (is.null(df_bystations)) {
      df_bystations <- moyennes_par_station
    } else {
      df_bystations <- full_join(df_bystations, moyennes_par_station, by = "id")
    }

}
#}
# Afficher
print(head(df_bystations))

kmeans_stations <- kmeans(df_bystations, centers=4)
cat("Groupes des stations pour le premier K-means :\n")
print(kmeans_stations$cluster)

# AJOUT : Ajouter les clusters comme colonne
df_bystations$groupe <- kmeans_stations$cluster
# AJOUT : Joindre les coordonnées
df_bystations_geo <- dplyr::left_join(df_bystations, coord_stations, by = "id")
#   AJOUT : Palette de couleurs
library(RColorBrewer)
couleurs <- brewer.pal(4, "Set1")
4:45
# Affichage sur la carte
ggplot() +
  geom_polygon(data = france, aes(x = long, y = lat, group = group),
               fill = "lightgray", color = "white") +
  geom_point(data = df_bystations_geo, aes(x = lon, y = lat, color = factor(groupe)), size = 3) +
  scale_color_manual(values = couleurs) +
  theme_minimal() +
  coord_fixed(ratio = 1.2) +
  ggtitle("Carte des stations météo de France métropolitaine selon le clustering")